In [1]:
# !pip install torch torchvision matplotlib

In [1]:
import os
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

data_dir = "unData"
num_classes = 2
batch_size = 16
epochs = 6
device = torch.device('cpu')

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

dataset = datasets.ImageFolder(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
class_names = dataset.classes  

# Model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Acc: {epoch_acc:.4f}")

torch.save(model.state_dict(), "newfiber_model.pth")

c:\Users\unmes\anaconda3\envs\orient\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\unmes\anaconda3\envs\orient\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/6 - Loss: 0.5183 - Acc: 0.7750
Epoch 2/6 - Loss: 0.1220 - Acc: 0.9750
Epoch 3/6 - Loss: 0.0707 - Acc: 1.0000
Epoch 4/6 - Loss: 0.0535 - Acc: 0.9750
Epoch 5/6 - Loss: 0.0148 - Acc: 1.0000
Epoch 6/6 - Loss: 0.0060 - Acc: 1.0000


In [5]:
import os
from PIL import Image
import torch
from torchvision import transforms, models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 2
model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("newfiber_model.pth", map_location=device))
model.to(device)
model.eval()

class_names = ['bidirectional','unidirectional']

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x.convert("RGB")),
    transforms.ToTensor(),
])

In [6]:

def predict_folder(folder_path):
    results = {}
    for fname in os.listdir(folder_path):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        img_path = os.path.join(folder_path, fname)
        img = Image.open(img_path)
        img = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(img)
            _, pred = torch.max(output, 1)
        results[fname] = class_names[pred.item()]
    return results

# Example usage
data_dir = "unData"  # 
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
        print(f"Processing folder: {folder}")
        predictions = predict_folder(folder_path)
        for img_name, pred_class in predictions.items():
            print(f"{img_name}: {pred_class}")


Processing folder: bidirectional
12.png: bidirectional
14.png: bidirectional
87.png: bidirectional
9898.png: bidirectional
baad.png: bidirectional
bidu.png: bidirectional
i8y8qw.png: bidirectional
image copy.png: bidirectional
image.png: bidirectional
image1.png: bidirectional
image12.png: bidirectional
iojiji.png: bidirectional
kya.png: bidirectional
nahi.png: bidirectional
ok12.png: bidirectional
plpl.png: bidirectional
poque.png: bidirectional
q.png: bidirectional
qwe.png: bidirectional
uhhu.png: bidirectional
Processing folder: unidirectional
achha.png: unidirectional
bet.png: unidirectional
chalo.png: unidirectional
cof.png: unidirectional
high-strength-carbon-fabric-CTech-LLC.jpg: unidirectional
jack.png: unidirectional
koi.png: unidirectional
left.png: unidirectional
muh.png: unidirectional
mushk.png: unidirectional
nee.png: unidirectional
oggy.png: unidirectional
ohah.png: unidirectional
okok.png: unidirectional
oly.png: unidirectional
opohto.png: unidirectional
oye.png: unidir

In [ ]:
from PIL import Image
import torch

def predict_custom_image(image_path, model, transform, class_names, device):
    model.eval()
    img = Image.open(image_path)
    img = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(img)
        _, pred = torch.max(output, 1)
    return class_names[pred.item()]
    
print(f"Prediction: {predict_custom_image('angled.png', model, transform, class_names, device)} fibre orientation")
print(f"Prediction: {predict_custom_image('long.png', model, transform, class_names, device)} fibre orientation")
print(f"Prediction: {predict_custom_image('trans.png', model, transform, class_names, device)} fibre orientation")
print(f"Prediction: {predict_custom_image('cross.png', model, transform, class_names, device)} fibre orientation")

Prediction: angle_plied fibre orientation
Prediction: longitudinal fibre orientation
Prediction: transverse fibre orientation
Prediction: cross_plied fibre orientation
